<a href="https://colab.research.google.com/github/Tstrebe2/umich-mads-capstone-project/blob/tim-branch/tim-playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install pydicom -q

     |████████████████████████████████| 2.0 MB 4.8 MB/s 


In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/capstone/assets/stage_2_train_images.zip'

!unzip {path}

In [7]:
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.optim import Adam
from torch import nn
import pydicom as dicom

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [29]:
class CustomImageDataset(Dataset):
  def __init__(self, annotations_file, img_dir, indices=None, transform=None, target_transform=None):
    if indices:
      self.img_labels = pd.read_csv(annotations_file).iloc[indices]
    else:
      self.img_labels = pd.read_csv(annotations_file)

    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = ''.join([os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]), '.dcm'])
    
    image = dicom.dcmread(os.path.join(self.img_dir, img_path))
    image = image.pixel_array

    label = self.img_labels.iloc[idx, -1]

    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

  def get_classes(self):
    return self.img_labels.iloc[:, -1].unique()

  classes = property(get_classes)

In [9]:
from sklearn.model_selection import train_test_split

annotations_file = '/content/drive/MyDrive/capstone/assets/stage_2_train_labels.csv.zip'
img_dir = '/content'

label_df = pd.read_csv(annotations_file)

X_train, X_test = train_test_split(label_df, test_size=.7, stratify=label_df.Target, random_state=99)
X_train, X_val, = train_test_split(X_train, test_size=.6, stratify=X_train.Target, random_state=99)
X_train, X_test, X_val = X_train.index.tolist(), X_test.index.tolist(), X_val.index.tolist()

len(X_train), len(X_val),  len(X_test)

(3627, 5441, 21159)

In [83]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((256, 256)),
    torchvision.transforms.Normalize(0.5547, 0.1781),
    torchvision.transforms.RandomHorizontalFlip(.1),
    torchvision.transforms.RandomRotation(degrees=(-7, 7))
])

train_dataset = CustomImageDataset(annotations_file, img_dir, indices=X_train, transform=transform)
val_dataset = CustomImageDataset(annotations_file, img_dir, indices=X_val, transform=transform)
test_dataset = CustomImageDataset(annotations_file, img_dir, indices=X_test, transform=transform)

train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=32)
test_data_loader =  DataLoader(test_dataset, batch_size=32)

print(len(train_data_loader.dataset)//32)
print(len(val_data_loader.dataset)//32)
print(len(test_data_loader.dataset)//32)

113
170
661


In [93]:
import torchvision
import torch

class LeNet(torch.nn.Module):
  def __init__(self, num_channels, classes):
    super(LeNet, self).__init__()

    self.conv1 = torch.nn.Conv2d(in_channels=num_channels, out_channels=20, kernel_size=20)
    self.relu1 = torch.nn.ReLU()
    self.maxpool1 = torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

    self.conv2 = torch.nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5))
    self.relu2 = torch.nn.ReLU()
    self.maxpool2 = torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    
    self.fc1 = torch.nn.Linear(in_features=162450, out_features=500)
    self.relu3 = torch.nn.ReLU()
    
    self.fc2 = torch.nn.Linear(in_features=500, out_features=classes)
    self.logSoftmax = torch.nn.LogSoftmax(dim=1)

  def forward(self, x):

    x = self.conv1(x)
    x = self.relu1(x)
    x = self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x = self.maxpool2(x)

    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = self.relu3(x)

    x = self.fc2(x)
    output = self.logSoftmax(x)

    return output

init_learning_rate = 1e-3
batch_size = 64
epochs = 10

train_split = .25
val_split = 1-train_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LeNet(num_channels=1, classes=len(train_data_loader.dataset.classes)).to(device)

opt = Adam(model.parameters(), lr=init_learning_rate)
loss_fn = nn.NLLLoss()

In [96]:
for e in range(epochs):
  model.train()

  total_training_loss = 0
  total_val_loss = 0

  train_correct = 0
  val_correct = 0

  for x, y in train_data_loader:
    x, y = x.to(device), y.to(device)

    pred = model(x)
    loss = loss_fn(pred, y)

    opt.zero_grad()
    loss.backward()
    opt.step()

    total_training_loss += loss
    train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

In [100]:
print('Total training loss:', total_training_loss.detach())
print('Train correct:', train_correct)

Total training loss: tensor(118.8102, device='cuda:0')
Train correct: 2529.0


In [97]:
with torch.no_grad():
  model.eval()

  for (x, y) in val_data_loader:
    (x, y) = (x.to(device), y.to(device))
    pred = model(x)
    total_val_loss += loss_fn(pred, y)
    val_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

In [102]:
print('Val training loss:', total_val_loss.detach())
print('Val correct:', val_correct)

Val training loss: tensor(94.5975, device='cuda:0')
Val correct: 3843.0


In [110]:
with torch.no_grad():
  model.eval()

  for (x, y) in val_data_loader:
    (x, y) = (x.to(device), y.to(device))
    pred = model(x)
    print(pred.argmax(1),'\n', y)
    break
    total_val_loss += loss_fn(pred, y)
    val_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0') 
 tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
